In [1]:
import os
import sys
import json
import requests
sys.path.insert(0, "../..")
from src.fetch.fetch_doc_lists import parse_file_list_docx

In [2]:
with open("../../data/processed/list_and_blanks/state_duty.json", "r", encoding="utf-8") as f:
    state_duty = json.load(f)
with open("../../data/processed/list_and_blanks/review_period.json", "r", encoding="utf-8") as f:
    review_period = json.load(f)

In [3]:
with open("../../data/processed/list_and_blanks/list_citizenship_separate_categories.json", "r", encoding="utf-8") as f:
    input_data = json.load(f)

for item in input_data:
    os.makedirs("../../data/raw/citizenship_separate_categories_raw", exist_ok=True)
    url = item["href"]
    filename = url.split("/")[-1]
    filepath = os.path.join("../../data/raw/citizenship_separate_categories_raw", filename)
    # 下载文件（若不存在）
    if not os.path.exists(filepath):
        r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, verify=False)
        r.raise_for_status()
        with open(filepath, "wb") as f:
            f.write(r.content)
        print(f"下载完成: {filename}")

In [4]:
results = {}
application_type = "Гражданство (отдельные категории)"
results[application_type] = []

with open("../../data/processed/list_and_blanks/list_citizenship_separate_categories_processed.json", "r", encoding="utf-8") as f:
    input_data = json.load(f)

for item in input_data:
    # 添加结果
    required_documents_list = parse_file_list_docx(filepath)
    results[application_type].append({
        "id": len(results[application_type]),
        "application_type": application_type,
        "text": item["text"],
        "required_documents_list": required_documents_list,
        "state_duty_law": state_duty[application_type]["law"],
        "receipt_form_payment": state_duty[application_type]["href"],
        "review_period": review_period[application_type]["review_period"]
    })

In [5]:
with open("../../data/processed/list_and_blanks/parsed_doc_lists.json", "r", encoding="utf-8") as f:
    parsed_doc_lists = json.load(f)
parsed_doc_lists.update(results)

# 保存结果
with open("../../data/processed/list_and_blanks/parsed_doc_lists.json", "w", encoding="utf-8") as f:
    json.dump(parsed_doc_lists, f, ensure_ascii=False, indent=2)

print("解析完成，结果已保存到 data/processed/list_and_blanks/parsed_doc_lists.json")

解析完成，结果已保存到 data/processed/list_and_blanks/parsed_doc_lists.json
